In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from ipywidgets import AppLayout
from ipywidgets import Dropdown, Layout, HTML, AppLayout, VBox, Label, HBox, BoundedFloatText, interact, Output
import plotly.graph_objects as go

In [2]:
import pandas as pd
from sklearn.datasets import load_iris

data = load_iris()

df = pd.DataFrame(data = data['data'], columns = data.feature_names)

species_dict = {0:'setosa', 1: 'versicolor', 2: 'virginica'}

df['species'] = [species_dict[x] for x in data.target]

# df.head()

In [3]:
# set default x and y axis values (these will be able to be changed via the widgets later)
x = 'sepal length (cm)'
y = 'sepal width (cm)'

# create df subsets by species for use in traces
df_setosa = df[df.species == 'setosa']
df_versicolor = df[df.species == 'versicolor']
df_virginica = df[df.species == 'virginica']

# create traces for each species
setosa = go.Scatter(x=df_setosa[x], y=df_setosa[y],
                    mode='markers',
                    name='setosa')
versicolor = go.Scatter(x=df_versicolor[x], y=df_versicolor[y],
                    mode='markers',
                    name='versicolor')
virginica = go.Scatter(x=df_virginica[x], y=df_virginica[y],
                    mode='markers',
                    name='virginica')

# create the graph object as a FigureWidget
g = go.FigureWidget(data=[setosa, versicolor, virginica],
                    layout=go.Layout(
                            title={
                                'text': "Iris Measurements",
                                'y':0.85,
                                'x':0.5,
                                'xanchor': 'center',
                                'yanchor': 'top'},
                            xaxis_title=x,
                            yaxis_title=y,
                            legend_title = "Iris Species"
                    ))

# g.show()

In [4]:
select_x = Dropdown(options = data.feature_names,
                           layout = Layout(width = '160px'))
select_y = Dropdown(options = data.feature_names[::-1],
                           layout = Layout(width = '160px'))

In [5]:
# this function specifies which values to update each time 
# a widget value is changed

def response(change):
    with g.batch_update():
        g.data[0].x = df_setosa[select_x.value]
        g.data[1].x = df_versicolor[select_x.value]
        g.data[2].x = df_virginica[select_x.value]

        g.data[0].y = df_setosa[select_y.value]
        g.data[1].y = df_versicolor[select_y.value]
        g.data[2].y = df_virginica[select_y.value]

        g.layout = go.Layout(
                            title={
                                'text': "Iris Measurements",
                                'y':0.85,
                                'x':0.5,
                                'xanchor': 'center',
                                'yanchor': 'top'},
                            xaxis_title=select_x.value,
                            yaxis_title=select_y.value,
                            legend_title = "Iris Species"
                    )

# the observe function will call the response function any time the 
# widget is updated
select_x.observe(response, names="value")
select_y.observe(response, names = "value")

In [6]:
container = HBox([select_x, select_y])
VBox([container,
              g])

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import pickle

features = ['sepal length (cm)' , 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
X_train, X_test, y_train, y_test = train_test_split(df[features], df.species, test_size = 0.25, stratify = df['species'], random_state = 42)

knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(X_train, y_train)

# pred = knn.predict(X_test)
# print('accuracy = ', metrics.accuracy_score(y_test, pred))
#   accuracy was 94.7%

# use pickle to save the model
with open('knn.pickle', 'wb') as to_write:
    pickle.dump(knn, to_write)

In [8]:
knn_model = pickle.load(open('knn.pickle', 'rb'))

In [9]:
sepal_length = BoundedFloatText(value = 8, min = 4.0, max = 8.0, step = 0.2, description = 'sepal length')
sepal_width = BoundedFloatText(value = 8, min = 1.8, max = 4.6, step = 0.2, description = 'sepal width')
petal_length = BoundedFloatText(value = 8, min = 0.8, max = 7, step = 0.2, description = 'petal length')
petal_width = BoundedFloatText(value = 8, min = 0.0, max = 2.8, step = 0.2, description = 'petal width')

In [10]:
def run_model(sepal_length, sepal_width, petal_length, petal_width):
    
    result = knn.predict([[sepal_length, sepal_width, petal_length, petal_width]])[0]
    
    out = Output(layout={'border': '1px solid white'})
    
    with out:
        display(HTML(value=f'Predicted species: <b>{result.capitalize()}</b>'))
        
    return out

In [11]:
interact(run_model,
        sepal_length = BoundedFloatText(value = 8, min = 4.0, max = 8.0, step = 0.2, description = 'sepal length'),
        sepal_width = BoundedFloatText(value = 8, min = 1.8, max = 4.6, step = 0.2, description = 'sepal width'),
        petal_length = BoundedFloatText(value = 8, min = 0.8, max = 7, step = 0.2, description = 'petal length'),
        petal_width = BoundedFloatText(value = 8, min = 0.0, max = 2.8, step = 0.2, description = 'petal width'),
        );

interactive(children=(BoundedFloatText(value=8.0, description='sepal length', max=8.0, min=4.0, step=0.2), Bou…